In [111]:
import os
import pandas as pd
import numpy as np
from tqdm import trange,tqdm

# constant, functions assignment:

In [160]:
dateCol=["Release Date","Customer Target XFD (CTXFD)","NB Requested XFD (NBRXFD)","Orig Fact Cfm XFD (OCXFD)","NB Curr Req XFD","Vendor Current Req XFD",
         "Curr Fact Cfm XFD (CCXFD)","SI Date","Start Production Dt","Customer Request Date","Original Customer Target XFD (OCTXFD)","Current Customer Target XFD (CCTXFD)",
         "Exp XFD","Booking Request Send Date","Production Complete Date","Intro Date (PPM)","Receipt Date"]

vsCol=["NB Requested XFD (NBRXFD)","Orig Fact Cfm XFD (OCXFD)","Customer Target XFD (CTXFD)","SI Date"]

rCol=vsCol[:]
rCol.extend([x for x in cj.columns if "vs" in x.lower() and "/" not in x])

lateRenameRule=dict(zip(['SI vs CTXFD', 'SI vs NBRXFD', 'SI vs OCXFD'],
["iLateds", "nLateds",  "cLateds", ]))

dfs=["cj", "ftv", "pwi", "pyv", "sa"]

# read

In [161]:
for i in [x for x in os.listdir(".") if x.endswith("csv")]:
    vName=i.lower().split(".")[0].replace("1","").replace("2","")
    # var creation- orderID, product, orderqua:
    globals()[vName]=pd.read_csv(i,thousands=",").rename({"Order#":"orderID","GBU (PPM)":"product","Quantity":"orderQnty"}, axis=1)

# dfs=[cj, ftv, pwi, pyv, sa]

C:\Users\JselZ\AppData\Local\Temp\ipykernel_8676\4057042712.py:4: DtypeWarning: Columns (62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[vName]=pd.read_csv(i,thousands=",").rename({"Order#":"orderID","GBU (PPM)":"product","Quantity":"orderQnty"}, axis=1)
C:\Users\JselZ\AppData\Local\Temp\ipykernel_8676\4057042712.py:4: DtypeWarning: Columns (51,63) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[vName]=pd.read_csv(i,thousands=",").rename({"Order#":"orderID","GBU (PPM)":"product","Quantity":"orderQnty"}, axis=1)
C:\Users\JselZ\AppData\Local\Temp\ipykernel_8676\4057042712.py:4: DtypeWarning: Columns (62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[vName]=pd.read_csv(i,thousands=",").rename({"Order#":"orderID","GBU (PPM)":"product","Quantity":"orderQnty"}, axis=1)


# preprocessing:

In [162]:
for name in dfs:
    df=globals()[name]
    df.loc[df[df["SI Month"]=="2022/00"].index,"SI Month"]="2021/12"
    df[dateCol]=df[dateCol].apply(pd.to_datetime)
    globals()[name]=df

# var creation:

In [163]:
for names in tqdm(["cj", "ftv", "pwi", "pyv", "sa"]):
    # styleseq:
    df=globals()[names]
    df["SI Month"] = pd.to_datetime(df["SI Month"], format='%Y/%m')
    df=df.sort_values(by=["Style","SI Month"])
    df['styleseq'] = df.groupby('Style')["SI Month"].rank(method='dense').astype(int)
    df["SI Month"] = df["SI Month"].dt.strftime('%Y/%m')

    # ontime:
    for i in zip(["SI vs CTXFD Early/On-time/Late","SI vs NBRXFD Early/On-time/Late", "SI vs OCXFD Early/On-time/Late"]
    ,["iOnTime","nOnTime","cOnTime"]):
        df[i[1]]=df[i[0]].map({'Early':3,'On-time':2,'Late':1})

    globals()[names]=df

100%|██████████| 5/5 [00:00<00:00,  6.83it/s]


In [164]:
# err. time cols. handling:
cj.loc[2004,[x for  x in cj.columns if "CTXFD" in x][:3]]="4/3/2022"
cj.loc[325,"NB Requested XFD (NBRXFD)"]="2021-03-09"
cj.loc[392,"NB Requested XFD (NBRXFD)"]="2021-03-16"

ftv.loc[788,"NB Requested XFD (NBRXFD)"]="2021/1/25"

pwi.loc[[7321, 7548, 7550, 7551, 7552, 7566, 7946], "NB Requested XFD (NBRXFD)"]="9/8/2021"
pwi.loc[22657, "Orig Fact Cfm XFD (OCXFD)"]="1/10/2023"

pyv.loc[378, "Customer Target XFD (CTXFD)"]="1/18/2021"

sa.loc[11,"NB Requested XFD (NBRXFD)"]="1/11/2021"
sa.loc[13,"NB Requested XFD (NBRXFD)"]="1/4/2021"
sa.loc[[12526,12527],"NB Requested XFD (NBRXFD)"]="12/13/2022"

# recalculating vs values:
for df in dfs:
    df=globals()[name]
    for group in zip(rCol[:3],rCol[-3:]):
        df[group[-1]]=(pd.to_datetime(df["SI Date"])-pd.to_datetime(df[group[0]])).dt.days 
    globals()[name]=df

In [165]:
# ld_timei, ld_timen, ld_timec, prd_dayso, prd_timeo, introtime:
for name in dfs:
    df=globals()[name]
    df["ld_timei"]=(df["Customer Target XFD (CTXFD)"]-df["Release Date"]).dt.days #ideal
    df["ld_timen"]=(df["NB Requested XFD (NBRXFD)"]-df["Release Date"]).dt.days #brand
    df["ld_timec"]=(df["Orig Fact Cfm XFD (OCXFD)"]-df["Release Date"]).dt.days #confirm
    df["prd_dayso"]=(df["Production Complete Date"]-df["Start Production Dt"]).dt.days 
    df["prd_timeo"]=(df["Start Production Dt"]-df["Release Date"]).dt.days 
    df["introtime"]=(df["Start Production Dt"]-df["Intro Date (PPM)"]).dt.days 

    # iLateds, iOnTime, nLateds, nOnTime, cLateds, cOnTime:
    df.rename(lateRenameRule, axis=1, inplace=True)
    
    globals()[name]=df


In [166]:
l=[x for x in '''
orderID
product
styleseq
orderQnty
ld_timei
ld_timen
ld_timec
prd_dayso
prd_timeo
iLateds
iOnTime
nLateds
nOnTime
cLateds
cOnTime
introtime
'''.split("\n") if x!='']
for name in dfs:
    df=globals()[name]
    print (set(l).issubset(set(df.columns)), set(l)-set(df.columns))

True set()
True set()
True set()
True set()
True set()


In [167]:
for name in dfs:
    df=globals()[name]
    if name in ['cj','sa']:
        alias=name.upper()+"1"
    elif name in ['ftv','pwi']:
        alias=name.upper()+"2"
    else:
        alias=name.upper()
    df["factoryCode"]=alias
    globals()[name]=df

resultCols=['factoryCode']+l

In [168]:
result=pd.concat(
    [cj[resultCols],ftv[resultCols],pwi[resultCols],pyv[resultCols],sa[resultCols]],ignore_index=True, axis=0
)

result

,factoryCode,orderID,product,styleseq,orderQnty,ld_timei,ld_timen,ld_timec,prd_dayso,prd_timeo,iLateds,iOnTime,nLateds,nOnTime,cLateds,cOnTime,introtime
0,CJ1,7344924,Lifestyle,1,4704,151,148.0,148,13.0,134,-3,3,0.0,2.0,0,2,-72.0
1,CJ1,7344925,Lifestyle,1,4620,151,148.0,148,13.0,134,-3,3,0.0,2.0,0,2,-72.0
2,CJ1,7348863,Lifestyle,1,600,166,166.0,166,3.0,160,-3,3,-3.0,3.0,-3,3,-30.0
3,CJ1,7311119,Lifestyle,2,780,223,223.0,223,2.0,219,0,2,0.0,2.0,0,2,-21.0
4,CJ1,7351430,Lifestyle,2,5838,184,181.0,181,5.0,166,-11,3,-8.0,3.0,-8,3,-24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110316,SA1,7437837,Kids,8,180,13,6.0,6,3.0,1,-9,3,-2.0,3.0,-2,3,835.0
110317,SA1,7437838,Kids,8,180,13,6.0,6,2.0,1,-10,3,-3.0,3.0,-3,3,835.0
110318,SA1,7187783,Kids,1,240,80,82.0,82,33.0,52,5,2,3.0,1.0,3,1,806.0
110319,SA1,7158832,Kids,1,264,147,147.0,147,28.0,122,3,2,3.0,1.0,3,1,826.0


In [169]:
result.to_csv("orderCompleted.csv", index=False, encoding='utf-8')